In [20]:
import pandas as pd
import numpy as np
import math

In [2]:
links: pd.DataFrame = pd.read_csv('../DataSet/links.csv')
movies: pd.DataFrame = pd.read_csv('../DataSet/movies.csv')
ratings: pd.DataFrame = pd.read_csv('../DataSet/ratings.csv')
tags: pd.read_csv = pd.read_csv('../DataSet/tags.csv')

In [3]:
links.head()

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0


In [4]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [5]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [6]:
tags.head()

,userId,movieId,tag,timestamp
0,15,339,sandra 'boring' bullock,1138537770
1,15,1955,dentist,1193435061
2,15,7478,Cambodia,1170560997
3,15,32892,Russian,1170626366
4,15,34162,forgettable,1141391765


In [7]:
ratings.groupby('movieId')['rating'].mean()

movieId
1         3.872470
2         3.401869
3         3.161017
4         2.384615
5         3.267857
            ...   
161944    5.000000
162376    4.500000
162542    5.000000
162672    3.000000
163949    5.000000
Name: rating, Length: 9066, dtype: float64

In [8]:
ratings[ratings['movieId']==1]

,userId,movieId,rating,timestamp
495,7,1,3.0,851866703
699,9,1,4.0,938629179
889,13,1,5.0,1331380058
962,15,1,2.0,997938310
3105,19,1,3.0,855190091
...,...,...,...,...
98531,660,1,2.5,1436680062
98714,663,1,4.0,1438397999
98740,664,1,3.5,1362421730
99858,670,1,4.0,938782344


In [9]:
# tags.groupby('movieId').reset_index().columns.values
print(type(tags.groupby('movieId').mean()))

<class 'pandas.core.frame.DataFrame'>


### 电影字典 & 写入基本信息

In [10]:
mv = {} # key:movieId,value:信息字典
for i in range(movies.shape[0]):
    index = 

### 统计每个电影的tag

In [11]:
g = tags.groupby('movieId')
for k,v in g:
    # print(v)
    mvk = v.iloc[0,1]
    mvv = v.iloc[:,2].values
    # print(v.iloc[0,1],v.iloc[:,2].values)
    if mvk not in mv:
        mv[mvk]=dict()
    mv[mvk]['tag'] = list(mvv)
print(mv)

{1: {'tag': ['Pixar']}, 5: {'tag': ['steve martin']}, 47: {'tag': ['biblical', 'crime', 'dark', 'disturbing', 'greed', 'horror', 'serial killer', 'violent', 'psychology', 'twist ending']}, 50: {'tag': ['thriller']}, 104: {'tag': ['adam sandler']}, 111: {'tag': ['cult film', 'Martin Scorsese', 'social commentary']}, 150: {'tag': ['Ron Howard', 'tom hanks']}, 153: {'tag': ['Ei muista']}, 186: {'tag': ['hugh grant']}, 215: {'tag': ['ethan hawke', 'holes90s']}, 232: {'tag': ['aging', 'Ang Lee', 'cooking', 'food', 'relationships']}, 236: {'tag': ['meg ryan']}, 246: {'tag': ['school']}, 247: {'tag': ['stylized', 'surreal', 'surrealism', 'visceral']}, 260: {'tag': ['cult classic', 'Science Fiction', 'nerdy', 'Science Fiction', 'critically acclaimed', 'Science Fiction', 'awesome', 'awesome soundtrack', 'jedi', 'space adventure', 'coming of age', "space epic, science fiction, hero's journey", 'classic', 'sci-fi', 'supernatural powers', 'George Lucas', 'starwars', 'Science Fiction', 'space', 'cl

### 统计每个电影平均评分

In [12]:
g = ratings.groupby('movieId').mean()
indexes = g.index # 行号-索引对照
for i in range(g.shape[0]):
    # print(indexes[i],g.iloc[i,1])
    if indexes[i] not in mv:
        mv[indexes[i]] = dict()
    mv[indexes[i]]['meanRating'] = g.iloc[i,1]
print(mv)

{1: {'tag': ['Pixar'], 'meanRating': 3.8724696356275303}, 5: {'tag': ['steve martin'], 'meanRating': 3.267857142857143}, 47: {'tag': ['biblical', 'crime', 'dark', 'disturbing', 'greed', 'horror', 'serial killer', 'violent', 'psychology', 'twist ending'], 'meanRating': 4.034825870646766}, 50: {'tag': ['thriller'], 'meanRating': 4.370646766169155}, 104: {'tag': ['adam sandler'], 'meanRating': 3.2925531914893615}, 111: {'tag': ['cult film', 'Martin Scorsese', 'social commentary'], 'meanRating': 4.22457627118644}, 150: {'tag': ['Ron Howard', 'tom hanks'], 'meanRating': 3.9025}, 153: {'tag': ['Ei muista'], 'meanRating': 2.7829457364341086}, 186: {'tag': ['hugh grant'], 'meanRating': 2.875}, 215: {'tag': ['ethan hawke', 'holes90s'], 'meanRating': 3.7222222222222223}, 232: {'tag': ['aging', 'Ang Lee', 'cooking', 'food', 'relationships'], 'meanRating': 4.208333333333333}, 236: {'tag': ['meg ryan'], 'meanRating': 3.3583333333333334}, 246: {'tag': ['school'], 'meanRating': 4.040983606557377}, 24

### 统计每个电影的imdbId和tmdbId

In [21]:
g = links
for i in range(g.shape[0]):
    if math.isnan(g.iloc[i,1]) or math.isnan(g.iloc[i,2]):
        continue
    index = g.iloc[i,0]
    if index not in mv:
        mv[index] = dict()
    #print(g.iloc[i,1],g.iloc[i,2])
    mv[index]['imdbId'] = -1 if (g.iloc[i,1] == np.nan) else int(g.iloc[i,1])
    mv[index]['tmdbId'] = -1 if (g.iloc[i,2] == np.nan) else int(g.iloc[i,2])
print(mv)

{1: {'tag': ['Pixar'], 'meanRating': 3.8724696356275303, 'imdbId': 114709, 'tmdbId': 862}, 5: {'tag': ['steve martin'], 'meanRating': 3.267857142857143, 'imdbId': 113041, 'tmdbId': 11862}, 47: {'tag': ['biblical', 'crime', 'dark', 'disturbing', 'greed', 'horror', 'serial killer', 'violent', 'psychology', 'twist ending'], 'meanRating': 4.034825870646766, 'imdbId': 114369, 'tmdbId': 807}, 50: {'tag': ['thriller'], 'meanRating': 4.370646766169155, 'imdbId': 114814, 'tmdbId': 629}, 104: {'tag': ['adam sandler'], 'meanRating': 3.2925531914893615, 'imdbId': 116483, 'tmdbId': 9614}, 111: {'tag': ['cult film', 'Martin Scorsese', 'social commentary'], 'meanRating': 4.22457627118644, 'imdbId': 75314, 'tmdbId': 103}, 150: {'tag': ['Ron Howard', 'tom hanks'], 'meanRating': 3.9025, 'imdbId': 112384, 'tmdbId': 568}, 153: {'tag': ['Ei muista'], 'meanRating': 2.7829457364341086, 'imdbId': 112462, 'tmdbId': 414}, 186: {'tag': ['hugh grant'], 'meanRating': 2.875, 'imdbId': 113986, 'tmdbId': 11472}, 215: